In [1]:
import glob
import pandas as pd
from pathlib import Path
from PIL import Image
from xml.dom.minidom import Document
import os
import cv2
from pathlib import Path
from tqdm import tqdm

In [68]:
id2chi9 = {
    0: '鞋子',
    1: '包包',
    2: '上装_短款',
    3: '裤子',
    4: '半身裙',
    5: '连体装',
    6: '上装_长款',
    7: '超短裤',
    8: '超短裙',
}

id2eng9 = {
    0: 'shoe',  # 鞋子
    1: 'bag',  # 包包
    2: 'upper_short',  # 上装
    3: 'paints',  # 裤子
    4: 'skirt',  # 裙子
    5: 'wholebody',  # 连体装
    6: 'upper_long',
    7: 'paints_hot',
    8: 'miniskirt',
}

In [69]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [70]:
chi2id9 = get_list2dict(id2chi9.values(), id2chi9.keys())
eng2id9 = get_list2dict(id2eng9.values(), id2eng9.keys())

#### 数据加载

In [72]:
root_csv = Path('../../data/xml_csv/')
bboxes_df = pd.read_csv(root_csv /'gall7k_xml_20220715.csv')

In [73]:
bboxes_df['label9'] = bboxes_df['labelimg'].map(chi2id9)
bboxes_df['label9_eng'] = bboxes_df['label9'].map(id2eng9)
bboxes_df['label_xml'] = bboxes_df['label9_eng']

### pandas转xml

In [74]:
root = Path('/Users/chenyi/Desktop/check_img')
dataset_name = 'deploy'
txtPath = str(root /'labels') + '/'
picPath = str(root /'JPEGImages') + '/'
xmlPath = str(root /'xml') + '/'

In [9]:
pathes = glob.glob(txtPath+'*.txt')

In [8]:
# makexml(txtPath, xmlPath, picPath, dataset_name)

In [75]:
dataset = 'gallery2m_7k20220715'
root_save = Path(f'/Users/chenyi/Desktop/{dataset}')
bboxes_df['xml_tp'] = bboxes_df.apply(lambda x: '{0}/{1}/{2}.xml'.format(root_save, x['man'], x['stem']), axis=1)

In [76]:
group_dict = dict(list(bboxes_df.groupby('stem')))

In [77]:
bboxes_df.value_counts('labelimg'), len(bboxes_df.value_counts('labelimg'))

(labelimg
 鞋子       10008
 上装_短款     5623
 包包        3304
 裤子        2843
 上装_长款     1132
 半身裙       1024
 连体装        948
 超短裤        736
 超短裙        723
 dtype: int64,
 9)

In [80]:
for k, v in tqdm(group_dict.items()):
    stem = k
    rows = v.loc[:, ['width', 'height', 'label_xml', 'bx1', 'bx2', 'bx3', 'bx4', 'stem', 'xml_tp']].values
    xml_path = Path(rows[0, 8])
    if xml_path.is_file():
        continue
    if not xml_path.parent.is_dir():
        xml_path.parent.mkdir(parents=True)
    Pwidth, Pheight, Pdepth= rows[0,0], rows[0,1], 3
    xmlBuilder = Document()
    annotation = xmlBuilder.createElement("annotation")  # 创建annotation标签
    xmlBuilder.appendChild(annotation)
    flag = 0
    for row in rows:
        flag += 1
        label = row[2]
        xmin_num, ymin_num, xmax_num, ymax_num = row[3], row[4], row[5], row[6]
        folder = xmlBuilder.createElement("folder")  # folder标签
        folderContent = xmlBuilder.createTextNode(dataset)
        folder.appendChild(folderContent)
        annotation.appendChild(folder)
        
        if flag==1:
            filename = xmlBuilder.createElement("filename")  # filename标签
            filenameContent = xmlBuilder.createTextNode(stem+".jpg")
            filename.appendChild(filenameContent)
            annotation.appendChild(filename)

            size = xmlBuilder.createElement("size")  # size标签
            width = xmlBuilder.createElement("width")  # size子标签width
            widthContent = xmlBuilder.createTextNode(str(Pwidth))
            width.appendChild(widthContent)
            size.appendChild(width)
            height = xmlBuilder.createElement("height")  # size子标签height
            heightContent = xmlBuilder.createTextNode(str(Pheight))
            height.appendChild(heightContent)
            size.appendChild(height)
            depth = xmlBuilder.createElement("depth")  # size子标签depth
            depthContent = xmlBuilder.createTextNode(str(Pdepth))
            depth.appendChild(depthContent)
            size.appendChild(depth)
            annotation.appendChild(size)

        object = xmlBuilder.createElement("object")
        picname = xmlBuilder.createElement("name")
        nameContent = xmlBuilder.createTextNode(label)
        picname.appendChild(nameContent)
        object.appendChild(picname)
        pose = xmlBuilder.createElement("pose")
        poseContent = xmlBuilder.createTextNode("Unspecified")
        pose.appendChild(poseContent)
        object.appendChild(pose)
        truncated = xmlBuilder.createElement("truncated")
        truncatedContent = xmlBuilder.createTextNode("0")
        truncated.appendChild(truncatedContent)
        object.appendChild(truncated)
        difficult = xmlBuilder.createElement("difficult")
        difficultContent = xmlBuilder.createTextNode("0")
        difficult.appendChild(difficultContent)
        object.appendChild(difficult)
        bndbox = xmlBuilder.createElement("bndbox")
        
        xmin = xmlBuilder.createElement("xmin")
#         mathData=int(((float(oneline[1]))*Pwidth+1)-(float(oneline[3]))*0.5*Pwidth)
        mathData = xmin_num
        xminContent = xmlBuilder.createTextNode(str(mathData))
        xmin.appendChild(xminContent)
        bndbox.appendChild(xmin)
        
        ymin = xmlBuilder.createElement("ymin")
        mathData = ymin_num
#         mathData = int(((float(oneline[2]))*Pheight+1)-(float(oneline[4]))*0.5*Pheight)
        yminContent = xmlBuilder.createTextNode(str(mathData))
        ymin.appendChild(yminContent)
        bndbox.appendChild(ymin)
        
        xmax = xmlBuilder.createElement("xmax")
#         mathData = int(((float(oneline[1]))*Pwidth+1)+(float(oneline[3]))*0.5*Pwidth)
        mathData = xmax_num
        xmaxContent = xmlBuilder.createTextNode(str(mathData))
        xmax.appendChild(xmaxContent)
        bndbox.appendChild(xmax)
        
        ymax = xmlBuilder.createElement("ymax")
#         mathData = int(((float(oneline[2]))*Pheight+1)+(float(oneline[4]))*0.5*Pheight)
        mathData = ymax_num
        ymaxContent = xmlBuilder.createTextNode(str(mathData))
        ymax.appendChild(ymaxContent)
        bndbox.appendChild(ymax)
        object.appendChild(bndbox)

        annotation.appendChild(object)
        


    f = open(xml_path, 'w')
    xmlBuilder.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
    f.close()
#     break

100%|██████████| 7230/7230 [00:15<00:00, 477.47it/s]


In [12]:
def makexml(txtPath, xmlPath, picPath, dataset='VOC2007'):  # 读取txt路径，xml保存路径，数据集图片所在路径
#     files = os.listdir(txtPath)
    pathes = glob.glob(txtPath+'*.txt')[:10]
    files = [Path(x).name for x in pathes]
    for i, name in tqdm(enumerate(files)):
        xmlBuilder = Document()
        annotation = xmlBuilder.createElement("annotation")  # 创建annotation标签
        xmlBuilder.appendChild(annotation)
        txtFile = open(txtPath+name)
        txtList = txtFile.readlines()
        try:
            img = cv2.imread(picPath+name[0:-4]+".jpg")
            Pheight, Pwidth, Pdepth=img.shape
        except:
            img = cv2.imread(picPath+name[0:-4]+".jpeg")
            Pheight, Pwidth, Pdepth=img.shape

        flag = 0
        for i in txtList:
            flag += 1
            oneline = i.strip().split(" ")
            folder = xmlBuilder.createElement("folder")  # folder标签
            folderContent = xmlBuilder.createTextNode(dataset)
            folder.appendChild(folderContent)
            annotation.appendChild(folder)
            if flag == 1:
                filename = xmlBuilder.createElement("filename")  # filename标签
                filenameContent = xmlBuilder.createTextNode(name[0:-4]+".jpg")
                filename.appendChild(filenameContent)
                annotation.appendChild(filename)

                size = xmlBuilder.createElement("size")  # size标签
                width = xmlBuilder.createElement("width")  # size子标签width
                widthContent = xmlBuilder.createTextNode(str(Pwidth))
                width.appendChild(widthContent)
                size.appendChild(width)
                height = xmlBuilder.createElement("height")  # size子标签height
                heightContent = xmlBuilder.createTextNode(str(Pheight))
                height.appendChild(heightContent)
                size.appendChild(height)
                depth = xmlBuilder.createElement("depth")  # size子标签depth
                depthContent = xmlBuilder.createTextNode(str(Pdepth))
                depth.appendChild(depthContent)
                size.appendChild(depth)
                annotation.appendChild(size)

            object = xmlBuilder.createElement("object")
            picname = xmlBuilder.createElement("name")
            nameContent = xmlBuilder.createTextNode(dict[oneline[0]])
            picname.appendChild(nameContent)
            object.appendChild(picname)
            pose = xmlBuilder.createElement("pose")
            poseContent = xmlBuilder.createTextNode("Unspecified")
            pose.appendChild(poseContent)
            object.appendChild(pose)
            truncated = xmlBuilder.createElement("truncated")
            truncatedContent = xmlBuilder.createTextNode("0")
            truncated.appendChild(truncatedContent)
            object.appendChild(truncated)
            difficult = xmlBuilder.createElement("difficult")
            difficultContent = xmlBuilder.createTextNode("0")
            difficult.appendChild(difficultContent)
            object.appendChild(difficult)
            bndbox = xmlBuilder.createElement("bndbox")
            xmin = xmlBuilder.createElement("xmin")
            mathData=int(((float(oneline[1]))*Pwidth+1)-(float(oneline[3]))*0.5*Pwidth)
            xminContent = xmlBuilder.createTextNode(str(mathData))
            xmin.appendChild(xminContent)
            bndbox.appendChild(xmin)
            ymin = xmlBuilder.createElement("ymin")
            mathData = int(((float(oneline[2]))*Pheight+1)-(float(oneline[4]))*0.5*Pheight)
            yminContent = xmlBuilder.createTextNode(str(mathData))
            ymin.appendChild(yminContent)
            bndbox.appendChild(ymin)
            xmax = xmlBuilder.createElement("xmax")
            mathData = int(((float(oneline[1]))*Pwidth+1)+(float(oneline[3]))*0.5*Pwidth)
            xmaxContent = xmlBuilder.createTextNode(str(mathData))
            xmax.appendChild(xmaxContent)
            bndbox.appendChild(xmax)
            ymax = xmlBuilder.createElement("ymax")
            mathData = int(((float(oneline[2]))*Pheight+1)+(float(oneline[4]))*0.5*Pheight)
            ymaxContent = xmlBuilder.createTextNode(str(mathData))
            ymax.appendChild(ymaxContent)
            bndbox.appendChild(ymax)
            object.appendChild(bndbox)

            annotation.appendChild(object)

        f = open(xmlPath+name[0:-4]+".xml", 'w')
        xmlBuilder.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
        f.close()

#### Data copy